### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

In [2]:
school_data_complete

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635
...,...,...,...,...,...,...,...,...,...,...,...
39165,39165,Donna Howard,F,12th,Thomas High School,99,90,14,Charter,1635,1043130
39166,39166,Dawn Bell,F,10th,Thomas High School,95,70,14,Charter,1635,1043130
39167,39167,Rebecca Tanner,F,9th,Thomas High School,73,84,14,Charter,1635,1043130
39168,39168,Desiree Kidd,F,10th,Thomas High School,99,90,14,Charter,1635,1043130


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [3]:
#Calculations
Total_Schools = len(school_data_complete["School ID"].unique())
#Not using unique for counting students because some may have the same names
Total_Students = len(school_data_complete["student_name"])
#Find unique school id and the budget associated
total_budget = sum(school_data_complete["budget"].unique())
#Average and % Passing for Math and Reading
Average_Math = school_data_complete["math_score"].mean()
Average_Reading = school_data_complete["reading_score"].mean()
PassMath = (sum(school_data_complete["math_score"]>=70))/Total_Students *100
PassRead =(sum(school_data_complete["reading_score"]>=70))/Total_Students *100

#Find people who passed reading, and then from those people, select the people that also passed math
pass_reading_df = school_data_complete[school_data_complete["reading_score"]>=70]
pass_readingmath_df = pass_reading_df[pass_reading_df["math_score"]>=70]
overallpassing = (len(pass_readingmath_df))/Total_Students*100

# #Put into Dataframe
District_Summary = pd.DataFrame({"Total Schools":Total_Schools,
                                "Total Students":Total_Students,
                                "Total Budget":total_budget,
                                "Average Math Score":Average_Math,
                                "Average Reading Score":Average_Reading,
                                "% Passing Math":PassMath,
                                "% Passing Reading":PassRead,
                                "% Overall Passing": overallpassing},index=[0])
District_Summary["Total Budget"] = District_Summary["Total Budget"].map("${:,.2f}".format)
District_Summary["Total Students"] = District_Summary["Total Students"].map("${:,}".format)
District_Summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"$39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [14]:
#group by school name
unique_School_grouped = school_data_complete.groupby(["school_name"])

#Need to use one of the original data because we are working on a string and there is no function that can do it.
#Set "school name" as index for the consolidated table
#only keep the "type" column from the original school data
schooltype = school_data.set_index(["school_name"])["type"]
#Set "schooltype" as DataFrame
schooltype_df = pd.DataFrame(schooltype)

#total students by school
s_total_students = unique_School_grouped.count()["Student ID"]
schooltype_df["Total Students"] = s_total_students

#Total School Budget
s_t_budget = school_data_complete.sum()["budget"]
schooltype_df["Total School Budget"] = s_t_budget

#per student budget
per_stu_bud = s_t_budget/s_total_students
schooltype_df["Per Student Budget"] = per_stu_bud

#Average Math Score
a_m_s = unique_School_grouped.mean()["math_score"]
schooltype_df["Average Math Score"] = a_m_s

#Average Reading Score
a_r_s = unique_School_grouped.mean()["reading_score"]
schooltype_df["Average Reading Score"] = a_r_s

#% Passing Math using the data we had from above question
#Step 1: Find out the number of students who passed Math
#Step 2: Group those students by school name
#Step 3: Count the number of students within the school
#Note: Should be counting the number so we use count() here
pass_math_school = school_data_complete[school_data_complete["math_score"]>=70].groupby(["school_name"]).count()
per_pass_math = (pass_math_school/s_total_students)*100
#schooltype_df["% Passing Math"] = per_pass_math

#% Passing Reading - 
#same step as we did with passing math despite we have the number of students who passed Reading from the above question so that we can use that directly.
pass_reading_school = pass_reading_df.groupby(["school_name"]).count()
per_pass_reading = (pass_reading_school/s_total_students)*100
#schooltype_df["% Passing Reading"] = per_pass_reading
pass_reading_school
#% Overall passing
# "pass_readingmath_df" from above counted students who passed both reading and Math
# So we use this variable and group by "school_name"
overallpassing_school = pass_readingmath_df.groupby(["school_name"]).count()
per_overall_school = (overallpassing_school/s_total_students)*100
#schooltype_df["% Overall Passing"] = per_overall_school

# #Add calculations to schooltype_df dataframe
# per_school_summary = pd.DataFrame({"x":unique_School_grouped,
#                                    "y":unique_School_grouped,
#                                    "TotalStudents" : s_total_students,
#                                    "TotalSchoolBudget":s_t_budget,
#                                    "PerStudentBudget":per_stu_bud,
#                                    "AverageMathScore":a_m_s,
#                                    "AverageReading Score":a_m_s,
#                                    "%PassingMath":per_pass_math,
#                                    "%PassingReading" :per_pass_reading,
#                                    "%OverallPassing":per_overall_school})
# per_school_summary

#schooltype_df

In [15]:
pass_reading_school

,Student ID,student_name,gender,grade,reading_score,math_score,School ID,type,size,budget
school_name,,,,,,,,,,
Bailey High School,4077,4077,4077,4077,4077,4077,4077,4077,4077,4077
Cabrera High School,1803,1803,1803,1803,1803,1803,1803,1803,1803,1803
Figueroa High School,2381,2381,2381,2381,2381,2381,2381,2381,2381,2381
Ford High School,2172,2172,2172,2172,2172,2172,2172,2172,2172,2172
Griffin High School,1426,1426,1426,1426,1426,1426,1426,1426,1426,1426
Hernandez High School,3748,3748,3748,3748,3748,3748,3748,3748,3748,3748
Holden High School,411,411,411,411,411,411,411,411,411,411
Huang High School,2372,2372,2372,2372,2372,2372,2372,2372,2372,2372
Johnson High School,3867,3867,3867,3867,3867,3867,3867,3867,3867,3867


,Bailey High School,Cabrera High School,Figueroa High School,Ford High School,Griffin High School,Hernandez High School,Holden High School,Huang High School,Johnson High School,Pena High School,...,Wilson High School,Wright High School,budget,gender,grade,math_score,reading_score,size,student_name,type
school_name,,,,,,,,,,,,,,,,,,,,,
Bailey High School,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cabrera High School,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Figueroa High School,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Ford High School,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Griffin High School,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Hernandez High School,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Holden High School,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Huang High School,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Johnson High School,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
unique_School_grouped.head()

In [ ]:
schooltype_df

In [ ]:
s_total_students

In [ ]:
schooltype

In [ ]:
unique_School_grouped

In [ ]:
school_type

In [ ]:
s_t_budget

In [ ]:
s_total_students

In [ ]:
per_stu_bud 

In [ ]:
a_m_s

## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

## Reading Score by Grade 

* Perform the same operations as above for reading scores

## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

## Scores by School Size

* Perform the same operations as above, based on school size.

## Scores by School Type

* Perform the same operations as above, based on school type